# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing (https://en.wikipedia.org/wiki/Laplace_smoothing).

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

# Big Mac - McDonald's

    

In [47]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [85]:
# Abrindo arquivos
base = pd.read_excel('tweets_bigmac.xlsx')
teste = pd.read_excel('tweets_bigmac.xlsx','Teste')

sim = base[base.Classificacao == 1]
nao = base[base.Classificacao == 0]
sim.head()

,Treinamento,Classificacao
0,@whiskeyredbruno fds.......... estava bue a co...,1
1,what e o big mac que já existe há 15 anos fds,1
4,"@bloodymaminha ao menos não foi o ""mac""donald´...",1
6,na espera do meu big mac ❤️🤰🏻,1
7,fiquei tão bem com o big mac bacon que já nem ...,1


In [223]:
# Filtro para DataFrame
def filtro(tweets,nome):
    limpo = []
    eliminar = ['/',':','\n','"',"'",')','(',';','#','//','.',',','?','!','...','_','*','rt','|','`']
    for li in tweets[nome]:
        linha = li.lower()
        for erro in eliminar:
            linha = linha.replace(erro,'')
        limpo.append(linha)
    return limpo

base_sim = filtro(sim,'Treinamento')
base_nao = filtro(nao,'Treinamento')

In [179]:
# Filtro para uma linha
def filtro_linha(tweetsx):
    limp = []
    eliminar = ['/',':','\n','"',"'",')','(',';','#','//','.',',','?','!','...','_','*','rt','|','`']
    linha = tweetsx.lower()
    for erro in eliminar:
        linha = linha.replace(erro,'')
    limp.append(linha)
    return limp

In [194]:
# SPLIT frases

def splitador(linha_do_data):
    juntas = linha_do_data.split(' ')
    for p in juntas:
        if '@' in p or 'http' in p or p == '':
            juntas.remove(p)
    return juntas

In [221]:
# Organiza
def contador(base_sim_e_nao):
    listao =[]
    for e in base_sim_e_nao:
        spl = splitador(e)
        for u in spl:
            listao.append(u)
    return listao

treino_sim = contador(base_sim)
ts = pd.DataFrame({'palavras':treino_sim})
ts1 = ts['palavras'].value_counts()

treino_nao = contador(base_nao)
tn = pd.DataFrame({'palavras':treino_nao})
tn1 = tn['palavras'].value_counts()

bs1 = pd.DataFrame({'quantidade':ts1}).reset_index()
bn1 = pd.DataFrame({'quantidade':tn1}).reset_index()

bs1.head(3)

,index,quantidade
0,mac,233
1,big,233
2,um,156


In [182]:
# calcula probabilidade de cada palavra ser relevante ou irrelevante

def prob_palavra(bsn,n_sn,total_palavras):
    bsn['probabilidade'] = ''
    for i in range(len(bsn)):
        bsn['probabilidade'][i] = ((bsn['quantidade'][i])+1) / (n_sn * total_palavras)
    return bsn

n_total = len(treino_sim)+len(treino_nao)
prob_rel = len(treino_sim) / n_total
prob_irel = len(treino_nao) / n_total

In [219]:
bs2 = prob_palavra(bs1,len(treino_sim),n_total)
bn2 = prob_palavra(bn1,len(treino_nao),n_total)
bs2.head(3)

C:\Users\Duarte\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,index,quantidade,probabilidade
0,big,233,2.05442e-05
1,mac,233,2.05442e-05
2,um,156,1.37839e-05


In [201]:
# calcula probabilidade de uma frase ou tweet ser relevante ou irrelevante

def calcular_frase(f, base_sn, probabilidade_de_ser_relevante_ou_irrelevante):
    frase = filtro_linha(f)
    frase_dividida = splitador(frase[0])
    mult = [probabilidade_de_ser_relevante_ou_irrelevante] # multiplicara cada item dessa lista para saber a prob da frase
    ind = 0
    for palavra in frase_dividida:
        for tabela in base_sn['index']:
            if palavra == tabela:
                mult.append(base_sn['probabilidade'][ind])
        ind +=1
    res = 1
    for y in mult:
        res = res*y
    return res

In [227]:
# Classifica e coloca em um dataframe junto com os classificados manualmente

def classifica(dataFrame,coluna):
    resposta = []
    tweet_original = dataFrame[coluna]
    for linha in dataFrame[coluna]:
        
        R = calcular_frase(linha,bs2,prob_rel)
        I = calcular_frase(linha,bn2,prob_irel)
        if R>I:
            resposta.append(1)
        elif R<I:
            resposta.append(0)

    ans = pd.DataFrame({'Tweets':tweet_original})
    ans['classificador'] = resposta
    ans['manual'] = dataFrame.Classificacao
    return ans

r = classifica(teste,'Teste')
print(r)


                                                Tweets  classificador  manual
0    rt @joyce_188: dia 10 de novembro vou ganhar u...              0       0
1    rt @matusfuck: big mac é o pior, gd nojo https...              1       1
2    @anagonzaga15 agora tem um muito melhor, big m...              1       1
3    rt @portalstylinson: "louis, nós ouvimos dizer...              1       0
4      que fome, alguém por obséquio me dá um big mac?              0       1
5    nesses 3 dias o máximo que eu comi foi um mioj...              0       1
6    dois big mac,um milk-shake,uber,uma água de um...              0       0
7    sou estudante em penafiel, no entanto fui ao m...              0       1
8    vou comer um pão com manteiga imaginando um bi...              0       1
9    mano eu juro q acabei de fazer um sanduba com ...              0       0
10                    big mac não decepciona nuuuuunca              1       1
11              amassei um big mac e ainda to com fome          

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [229]:
# Verificando
positivos_falsos = 0
positivos_verdadeiros = 0
negativos_verdadeiros = 0
negativos_falsos = 0

for t in range(len(r)):
    comp = (r['classificador'][t],r['manual'][t])
    if comp == (1,0):
        positivos_falsos += 1
    elif comp == (1,1):
        positivos_verdadeiros += 1
    elif comp == (0,0):
        negativos_verdadeiros += 1 
    elif comp == (0,1):
        negativos_falsos += 1 

PF = 100*positivos_falsos /len(r)
PV = 100*positivos_verdadeiros /len(r)
NV = 100*negativos_verdadeiros / len(r)
NF = 100*negativos_falsos / len(r)

print('O classificador acertou no total',PV+NV,'%')
print(' ')
print('Porcentagem de positivos falsos',PF,'%')
print('Porcentagem de positivos verdadeiros',PV,'%')
print('Porcentagem de negativos verdadeiros',NV,'%')
print('Porcentagem de negativos falsos',NF,'%')

O classificador acertou no total 43.0 %
 
Porcentagem de positivos falsos 12.5 %
Porcentagem de positivos verdadeiros 32.5 %
Porcentagem de negativos verdadeiros 10.5 %
Porcentagem de negativos falsos 44.5 %


___
## Concluindo

Escreva aqui a sua conclusão.<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.<br />
Explique como são tratadas as mensagens com dupla negação e sarcasmo.<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).


# Conclusão


    Olhando para nossos resultados podemos observar que o algoritmo mesmo não tendo uma tendência tão grande a classificar mais vezes irrelevante do que relevante, a qualidade das suas classificações deixa a desejar, errando mais vezes quando decide que é irrelevante, esse efeito deve ter sido resultado de tweets que não tinham relevância terem conteúdos mais dispersos, enquanto os considerados relevantes convergem para um padrão mais ligado com o tema. Outro efeito que o classificador deve ter sofrido foi a divergência na classificação, onde um membro do grupo decidiu quais tweets seriam relevantes no treinamento e outro para o teste.
    Não foi implementado um sistema capas de analisar duplas negações e sarcasmo tendo em vista que o projeto por enquanto só classifique como relevante e irrelevante, e essas duas percepções fazem mais sentido se existisse subcategorias mais especificas, sendo as próximas iterações.
    


